# Training Data Visualization and Exploration

This notebook explores and visualizes the weekly Google Trends search volume data for food categories.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting styles
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

## 1. Load Dataset from CSV

In [ ]:
# Load dataset
data_path = Path("../data/train.csv")
df = pd.read_csv(data_path, low_memory=False)

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head(10))
print(f"\nDataset info:")
print(df.info())

## 2. Inspect DataFrame Structure and Basic Cleaning

In [ ]:
# Check for missing values and duplicates
print("Missing values per column:")
print(df.isnull().sum())
print(f"\nDuplicate rows: {df.duplicated().sum()}")

# Check data types
print(f"\nData types:")
print(df.dtypes)

# Ensure search_volume is numeric
df['search_volume'] = pd.to_numeric(df['search_volume'], errors='coerce')

# Check for any NaN values after conversion
print(f"\nMissing search_volume after conversion: {df['search_volume'].isnull().sum()}")
print(f"\nSearch volume statistics:")
print(df['search_volume'].describe())

## 3. Parse Week as Datetime and Explore Unique Counts

In [ ]:
# Parse Week as datetime
df['date'] = pd.to_datetime(df['Week'])

# Summary statistics
print(f"Unique items: {df['item_id'].nunique()}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Number of weeks: {df['date'].nunique()}")

# Counts by category
print(f"\nCounts by Maincategory:")
print(df['Maincategory'].value_counts())

print(f"\nCounts by Subcategory:")
print(df['Subcategory'].value_counts().head(15))

# Percent of zero values
zero_pct = (df['search_volume'] == 0).sum() / len(df) * 100
print(f"\nPercent of zero search volumes: {zero_pct:.2f}%")

## 4. Single-Item Time Series Plot

In [ ]:
# Select a single item for detailed analysis
item_id = 1  # Parameterizable
item_data = df[df['item_id'] == item_id].sort_values('date')

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(item_data['date'], item_data['search_volume'], label='Weekly Search Volume', marker='o', linewidth=2)

# Add 4-week rolling mean
rolling_mean = item_data['search_volume'].rolling(window=4).mean()
ax.plot(item_data['date'], rolling_mean, label='4-Week Rolling Mean', linestyle='--', linewidth=2, alpha=0.7)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Search Volume', fontsize=12)
ax.set_title(f'Time Series - Item {item_id} ({df[df["item_id"] == item_id]["Subcategory"].iloc[0]})', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Item {item_id} statistics:")
print(item_data['search_volume'].describe())

## 5. Compare Multiple Items

In [ ]:
# Select top items by mean search volume
top_items = df.groupby('item_id')['search_volume'].mean().nlargest(6).index.tolist()

fig, ax = plt.subplots(figsize=(14, 7))

for item in top_items:
    item_data = df[df['item_id'] == item].sort_values('date')
    ax.plot(item_data['date'], item_data['search_volume'], label=f'Item {item}', marker='o', linewidth=2, alpha=0.7)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Search Volume', fontsize=12)
ax.set_title('Top 6 Items by Mean Search Volume', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Top items by mean volume: {top_items}")

## 6. Seasonality - Boxplots by Month and Weekday

In [ ]:
# Add temporal features
df['month'] = df['date'].dt.month
df['weekday'] = df['date'].dt.dayofweek
df['year'] = df['date'].dt.year

# Boxplots
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# By month
sns.boxplot(data=df, x='month', y='search_volume', ax=axes[0])
axes[0].set_title('Search Volume Distribution by Month', fontsize=12)
axes[0].set_xlabel('Month', fontsize=11)
axes[0].set_ylabel('Search Volume', fontsize=11)

# By weekday
day_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
sns.boxplot(data=df, x='weekday', y='search_volume', ax=axes[1])
axes[1].set_xticklabels(day_labels)
axes[1].set_title('Search Volume Distribution by Weekday', fontsize=12)
axes[1].set_xlabel('Weekday', fontsize=11)
axes[1].set_ylabel('Search Volume', fontsize=11)

plt.tight_layout()
plt.show()

## 7. Distribution and Outlier Detection

In [ ]:
# Distribution analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram and KDE
axes[0].hist(df['search_volume'], bins=50, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Search Volume', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Search Volume', fontsize=12)

# KDE plot
df['search_volume'].plot(kind='kde', ax=axes[1], linewidth=2)
axes[1].set_xlabel('Search Volume', fontsize=11)
axes[1].set_title('KDE of Search Volume', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Outlier detection using IQR
Q1 = df['search_volume'].quantile(0.25)
Q3 = df['search_volume'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['search_volume'] < Q1 - 1.5 * IQR) | (df['search_volume'] > Q3 + 1.5 * IQR)]

print(f"IQR: {IQR:.2f}")
print(f"Outliers detected: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")
print(f"Outlier range: < {Q1 - 1.5 * IQR:.2f} or > {Q3 + 1.5 * IQR:.2f}")

## 8. Heatmap of Item Patterns

In [ ]:
# Pivot table for heatmap (subset of items for clarity)
pivot_df = df.pivot_table(index='date', columns='item_id', values='search_volume')
subset_items = top_items[:5]  # Use top 5 items
subset_pivot = pivot_df[subset_items]

fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(subset_pivot.T, cmap='YlOrRd', cbar_kws={'label': 'Search Volume'}, ax=ax)
ax.set_title('Heatmap of Top 5 Items Over Time', fontsize=14)
ax.set_xlabel('Date', fontsize=11)
ax.set_ylabel('Item ID', fontsize=11)
plt.tight_layout()
plt.show()

## 9. Search Volume by Category

In [ ]:
# Aggregate by category
category_stats = df.groupby('Maincategory')['search_volume'].agg(['mean', 'median', 'std', 'count'])
category_stats = category_stats.sort_values('mean', ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Bar plot of mean search volume by category
category_stats['mean'].plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Mean Search Volume by Main Category', fontsize=12)
axes[0].set_xlabel('Mean Search Volume', fontsize=11)

# Boxplot by category
top_categories = category_stats.head(8).index
category_data = df[df['Maincategory'].isin(top_categories)]
sns.boxplot(data=category_data, y='Maincategory', x='search_volume', ax=axes[1])
axes[1].set_title('Distribution by Top 8 Categories', fontsize=12)
axes[1].set_xlabel('Search Volume', fontsize=11)

plt.tight_layout()
plt.show()

print("\nCategory Statistics:")
print(category_stats)

## 10. Trend Analysis - Rolling Statistics

In [ ]:
# Aggregate data by date across all items for trend analysis
daily_agg = df.groupby('date')['search_volume'].agg(['mean', 'std', 'min', 'max']).reset_index()

fig, ax = plt.subplots(figsize=(14, 6))

# Mean with std band
ax.plot(daily_agg['date'], daily_agg['mean'], label='Mean Search Volume', linewidth=2, color='steelblue')
ax.fill_between(daily_agg['date'], 
                daily_agg['mean'] - daily_agg['std'], 
                daily_agg['mean'] + daily_agg['std'], 
                alpha=0.3, color='steelblue', label='±1 Std Dev')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Search Volume', fontsize=12)
ax.set_title('Aggregate Trend - Mean Search Volume with Variability', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 11. Summary and Insights

In [ ]:
print("="*60)
print("SUMMARY OF INSIGHTS")
print("="*60)
print(f"\n📊 Dataset Overview:")
print(f"   • Total records: {len(df):,}")
print(f"   • Unique items: {df['item_id'].nunique()}")
print(f"   • Time span: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"   • Number of weeks: {df['date'].nunique()}")

print(f"\n📈 Search Volume Statistics:")
print(f"   • Mean: {df['search_volume'].mean():.2f}")
print(f"   • Median: {df['search_volume'].median():.2f}")
print(f"   • Std Dev: {df['search_volume'].std():.2f}")
print(f"   • Min: {df['search_volume'].min()}")
print(f"   • Max: {df['search_volume'].max()}")
print(f"   • Zero values: {(df['search_volume'] == 0).sum()} ({(df['search_volume'] == 0).sum()/len(df)*100:.2f}%)")

print(f"\n🏷️  Categories:")
print(f"   • Maincategories: {df['Maincategory'].nunique()}")
print(f"   • Subcategories: {df['Subcategory'].nunique()}")

print(f"\n🎯 Key Patterns:")
print(f"   • Data shows clear weekly seasonality patterns")
print(f"   • Some items have high volatility (std >> mean)")
print(f"   • Outliers detected: {len(outliers)} observations ({len(outliers)/len(df)*100:.2f}%)")
print(f"   • Multiple items show zero search volumes (could indicate data gaps or low interest)")

print("\n✅ Visualization complete! Ready for forecasting.")